## Problem Statement - 
Increase in customer chat pendency due to high volume  
### Code Functionality- 
Take input for the pending chats for customers, depending on the channel the customer was redirected from, check customer selection, cuurent status for the order and raise appropriate tasks

### Input the file containing raised issues

In [271]:
import pymysql.cursors
import pandas as pd

# Mention file name and path for Line ID mapped file
incidetails=pd.read_excel("File.xlsx")

packets = incidetails[(incidetails['Leg']=='Forward')&(~incidetails['lineId'].isnull())]['lineId']
packets = packets.dropna(how = 'all')
p = ','.join(['\''+str(int(i))+'\'' for i in list(packets.unique())])

### Extract further details and check the current status for the issues

In [274]:
import pymysql.cursors

# Extract details for the line ids
Forward_details_lineid="""
select
's'+ol.store_order_id as store_order_id,
ol.id as entity,
ol.packet_id_fk,
ol.courier_code,
ol.packed_on,
ol.shipped_on,
cast(ol.created_on as date) as created_on,
ol.cancelled_on,
ol.status_code,
zipcode,
max(APD) as APD,
max(EPD) as EPD,
max(ESD) as ESD,
max(ol.delivered_on) as delivered_date
from order_line ol 
left join (select 
cast((case when orai.key in ('Actual_PROMISE_TIME') then value end) as date) as APD,
cast((case when orai.key in ('EXPECTED_PACKING_TIME') then value end) as date) as EPD,
cast((case when orai.key in ('EXPECTED_CUTOFF_TIME') then value end) as date) as ESD,
order_release_id_fk
from  order_release_additional_info orai
where order_release_id_fk in (select order_release_id_fk from order_line where id in ( """+p+""")
and orai.key in ('Actual_PROMISE_TIME','EXPECTED_PACKING_TIME','EXPECTED_CUTOFF_TIME')
)
) orai
on ol.order_release_id_fk=orai.order_release_id_fk
left join order_release_VIEW ord
on ol.store_order_id=ord.store_order_id
where ol.id in ( """+p+""") 
group by 1,2,3,4,5,6,7,8,9
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='charset',
                            cursorclass=pymysql.cursors.DictCursor)
Forward_details_lineid_df=pd.read_sql(Forward_details_lineid,connection)



C:\Users\300068032\AppData\Local\Continuum\anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1292, "Truncated incorrect DOUBLE value: 's'")
  self._do_get_result()


In [275]:
packets = Forward_details_lineid_df.packet_id_fk
packets = packets.dropna(how = 'all')
p = ','.join(['\''+str(int(i))+'\'' for i in list(packets.unique())])

In [276]:
# Extract failed attempts and loaction information for the line ids
Forward_location="""
select ots.order_id, som.status,zipcode,
max(failed_attempts) as failed_attempts,
max(dc_inscanned_date) as dc_inscanned_date
from order_to_ship ots
left join 
	(
		select order_id, status,
		max(last_modified_on) as last_modified_on
		from shipment_order_map 
		where order_id in ("""+p+""")
		group by 1
	) som
on ots.order_id=som.order_id
left join 
	(
		select order_id, failed_attempts
		from order_tracking ot
		where order_id in ("""+p+""")
	)ot
on ots.order_id=ot.order_id
left join 
	(
		select source_reference_id , max(received_date) as dc_inscanned_date
		from ml_shipment mls
		where source_reference_id in ("""+p+""")
        group by 1
	)mls
on ots.order_id=mls.source_reference_id 
where ots.order_id in ("""+p+""")
group by 1,2,3
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='prwrd,                             
                            db='db',
                            charset='charset',
                            cursorclass=pymysql.cursors.DictCursor)
Forward_location_df=pd.read_sql(Forward_location,connection)

In [277]:
Forward_location_df['order_id']=Forward_location_df['order_id'].fillna(0).astype(np.int64).astype(str)

In [278]:
Forward_details_lineid_df['packet_id_fk']=Forward_details_lineid_df['packet_id_fk'].fillna(0).astype(np.int64).astype(str)
Forward_concat_1=pd.merge(Forward_details_lineid_df,Forward_location_df, how='left', left_on='packet_id_fk', right_on='order_id')

In [279]:
from datetime import date 

Forward_concat_1['EPD'] = pd.to_datetime(Forward_concat_1['EPD'], format='%Y-%m-%d')
Forward_concat_1['APD'] = pd.to_datetime(Forward_concat_1['APD'], format='%Y-%m-%d')
Forward_concat_1['ESD'] = pd.to_datetime(Forward_concat_1['ESD'], format='%Y-%m-%d')

In [280]:
Forward_concat_1['failed_attempts']=Forward_concat_1['failed_attempts'].fillna(0)

### Logi for Task generation basis individual cases

In [281]:
# Task Generation Logic
Forward_concat_1.loc[((Forward_concat_1['status_code']=='S')
                    &(Forward_concat_1['failed_attempts']<1)
                    &(Forward_concat_1['courier_code']=='ML')
                    &(date.today()<=Forward_concat_1['APD'])
			)
                ,'Task_Status'
    
] = 'Customer_Awareness_Anxiety'

Forward_concat_1.loc[((Forward_concat_1['status_code'].isin(['WP','Q','RFR']))
                    
                    &(Forward_concat_1['failed_attempts']<1)
                    &(date.today()<=Forward_concat_1['EPD'])
			)
                ,'Task_Status'
    
] = 'Customer_Awareness_Anxiety'

Forward_concat_1.loc[((Forward_concat_1['status_code']=='PK')
                     &(Forward_concat_1['failed_attempts']<1)
                    &(date.today()<=Forward_concat_1['ESD'])
			)
                ,'Task_Status'
    
] = 'Customer_Awareness_Anxiety'


Forward_concat_1.loc[((Forward_concat_1['status_code'].isin(['WP','Q','RFR']))
                    &(date.today()>Forward_concat_1['EPD'])
			)
                ,'Task_Status'
    
] = 'Delay_In_Packing'

Forward_concat_1.loc[((Forward_concat_1['status_code']=='PK')
                    &(date.today()>Forward_concat_1['ESD'])
			)
                ,'Task_Status'
    
] = 'Delay_In_Shipping'

Forward_concat_1.loc[((Forward_concat_1['status_code']=='S')
                    &(date.today()>Forward_concat_1['APD'])
                    &(Forward_concat_1['dc_inscanned_date'].isnull())
                    &(Forward_concat_1['courier_code']=='ML')
			)
                ,'Task_Status'
    
] = 'Delayed_delivery_Not_reached_DC_ML'

Forward_concat_1.loc[((Forward_concat_1['status_code']=='S')
                    &(date.today()>Forward_concat_1['APD'])
                    &(~Forward_concat_1['dc_inscanned_date'].isnull())
                    &(Forward_concat_1['courier_code']=='ML')
			)
                ,'Task_Status'
    
] = 'Delayed_delivery_reached_DC_ML'

Forward_concat_1.loc[((Forward_concat_1['status_code']=='S')
                    &(date.today()>Forward_concat_1['APD'])
                    &(Forward_concat_1['courier_code']!='ML')
			)
                ,'Task_Status'
    
] = 'Delayed_delivery_3PL'




C:\Users\300068032\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  
C:\Users\300068032\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
C:\Users\300068032\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In

In [282]:
incidetails['lineId']=incidetails['lineId'].fillna(0).astype(np.int64).astype(str)
Forward_concat_1['entity']=Forward_concat_1['entity'].fillna(0).astype(np.int64).astype(str)

In [283]:
Forward_details_1=pd.merge(incidetails,Forward_concat_1, how='left', left_on='lineId', right_on='entity')

Forward_details_1.to_csv('File name')